# Environment setup

In [1]:
!npm install vietnamese-stopwords

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ vietnamese-stopwords@0.0.2
updated 1 package and audited 1 package in 0.602s
found 0 vulnerabilities



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd '/content/drive/MyDrive/CustomerReviewSentiment'

/content/drive/MyDrive/CustomerReviewSentiment


In [4]:
!pip install sentence_transformers

# Data preparation

In [5]:
import pandas as pd

data = pd.read_csv('clean_data.csv')

In [6]:
data

,Unnamed: 0,content,score,thumbsUpCount,Application
0,0,"rất thuận tiện , có lợi ích .",5,2,Messenger
1,1,tốt,4,213,Messenger
2,2,ok,5,1,Messenger
3,3,"xài rất tốt , nên tải nha",5,8,Messenger
4,4,gútttt,5,1,Messenger
...,...,...,...,...,...
310741,99995,ok,5,0,Instagram
310742,99996,g o o d,5,0,Instagram
310743,99997,có,5,0,Instagram
310744,99998,nhiều lúc tôi k lấy đc hiệu ứng,1,0,Instagram


In [7]:
print(data.columns[0])
data = data.drop(data.columns[0], axis=1)
data


Unnamed: 0


,content,score,thumbsUpCount,Application
0,"rất thuận tiện , có lợi ích .",5,2,Messenger
1,tốt,4,213,Messenger
2,ok,5,1,Messenger
3,"xài rất tốt , nên tải nha",5,8,Messenger
4,gútttt,5,1,Messenger
...,...,...,...,...
310741,ok,5,0,Instagram
310742,g o o d,5,0,Instagram
310743,có,5,0,Instagram
310744,nhiều lúc tôi k lấy đc hiệu ứng,1,0,Instagram


#Text Preprocessing with NLTK

In [8]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

nltk.download('wordnet')

def read_stop_words(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        stop_words = [line.strip() for line in file]
    return stop_words

stop_words_path = 'vietnamese-stopwords.txt'
stop_words = set(read_stop_words(stop_words_path))

lemmatizer = WordNetLemmatizer()

def clean_text(text):
    if not isinstance(text, str):
        return ''

    text = re.sub(r'[^\w\s]', '', text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

data['content'] = data.content.apply(lambda x: clean_text(x))


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


KeyboardInterrupt: ignored

In [ ]:
def count_sentences(text):
    sentences = text.split(".")
    return len(sentences)

data['sentence_count'] = data['content'].apply(count_sentences)
data = data[data['sentence_count'] <= 1].drop(columns=['sentence_count'])
data

#Feature Extraction

In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd
encode_model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')
encoded_values = encode_model.encode(list(data['content']), batch_size = 1280)
encoded_df = pd.DataFrame(encoded_values, columns=[f'feature_{i}' for i in range(encoded_values.shape[1])])
data = pd.concat([data, encoded_df], axis=1)
data

In [ ]:
data

In [ ]:
data_features = data.drop(columns=['score', 'thumbsUpCount', 'content']).values
data_features.shape

#Splitting the Dataset

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_features, data['score'], test_size=0.2, random_state=42)

#Training the XGBoost Classifier

In [ ]:
import xgboost as xgb

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)

model = xgb.XGBClassifier(max_depth=10, n_estimators=1000, learning_rate=0.01)
model.fit(X_train, y_train)

#Making Predictions and Evaluating the Model

In [ ]:
predictions = model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions))

In [ ]:
from sklearn.metrics import roc_auc_score
y_test_encoded = le.fit_transform(y_test)
pred_prob = model.predict_proba(X_test)
print(y_test_encoded.shape)
print(predictions.shape)
auc = roc_auc_score(y_test_encoded, pred_prob, multi_class='ovr')
print('AUC: %.2f' % auc)
